# Domain Scrape

In [1]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

In [2]:
headers = {"User-Agent":
           "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# home url of domian.com australia
home_url = "https://www.domain.com.au"

In [3]:
#postcode of properties
postcode_list = [i for i in range(3000,3100)]

In [4]:
#return unique extracted link for given postcode
def link_by_postcode(postcode):
    
    # list to store all the urls of properties
    list_of_links = []
    
    # store unique link
    abc_links = [] 
    
    # number of pages of search result are 50
    page_numbers = list(range(50))[1:50]
    
    # for loop for all 50 search(melbourne region) pages
    for page in page_numbers:

        # extracting html document of search page
        url = home_url + f"/rent/melbourne-vic-{postcode}/?excludedeposittaken=1&page={page}"

        # parsing html document to 'lxml' format
        bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

        #for each page
        # finding all the links available in 'ul' tag whos 'data-testid' is 'results'
        all_links = bs_object.find(
            "ul", {"data-testid": "results"}).findAll("a", href=re.compile("https://www.domain.com.au/*"))

        # inner loop to find links inside each property page because few properties are project so they have more properties inside their project page
        for link1 in all_links:
            # checking if it is a project and then performing similar above
            if 'project' in link1.attrs['href']:
                inner1_url = link1.attrs['href']
                inner1_bsobj = BeautifulSoup(requests.get(inner1_url, headers=headers).text, "html.parser")
                for link2 in inner1_bsobj.find("div", {"name": "listing-details__other-listings"}).findAll("a", href=re.compile("https://www.domain.com.au/*")):
                    if 'href' in link2.attrs:
                        list_of_links.append(link2.attrs['href'])
            else:
                list_of_links.append(link1.attrs['href'])
   
    # remove deplicated links
    for i in list_of_links: 
        if i not in abc_links: 
            abc_links.append(i) 
    return abc_links


In [5]:
links_test =link_by_postcode(3000)

In [6]:
# defining required regural expression for data extraction     
pattern = re.compile(r'>(.+)<!.*>(.+?)</span>.*')
pattern1 = re.compile(r'>(.+)<.')
pattern2 = re.compile(r'destination=(.+)" rel=.')
#school
pattern_school = re.compile(r'>(.+)<')
pattern_distance = re.compile(r'">(.+)<!')
#street information
pattern_address1 = re.compile(r'>(.+)<!.*>')
pattern_address2 = re.compile(r'>(.+)<.')

In [7]:
links_test

['https://www.domain.com.au/carpark-442-elizabeth-street-melbourne-vic-3000-15858295?topspot=1',
 'https://www.domain.com.au/3405-288-spencer-street-melbourne-vic-3000-16067117',
 'https://www.domain.com.au/1007-238-flinders-st-melbourne-vic-3000-16066003',
 'https://www.domain.com.au/1211-260-spencer-street-melbourne-vic-3000-16065974',
 'https://www.domain.com.au/504-350-la-trobe-street-melbourne-vic-3000-15302782',
 'https://www.domain.com.au/1106-555-flinders-street-melbourne-vic-3000-16064379',
 'https://www.domain.com.au/2901-200-spencer-street-melbourne-vic-3000-16064243',
 'https://www.domain.com.au/5401-464-collins-street-melbourne-vic-3000-16063884',
 'https://www.domain.com.au/405-39-queen-street-melbourne-vic-3000-16063772',
 'https://www.domain.com.au/113-350-st-kilda-road-melbourne-vic-3004-16063679',
 'https://www.domain.com.au/625-118-franklin-street-melbourne-vic-3000-16063139',
 'https://www.domain.com.au/503-8-bowen-crescent-melbourne-vic-3004-16062867',
 'https://ww

In [8]:
basic_feature_list = []


# loop to iterate through each url
for link in links_test[:10]:
    
    # opening urls
    bs_object = BeautifulSoup(requests.get(link, headers=headers).text, "html.parser")
    
    # extracting address/name of property
    property_name = bs_object.find("h1", {"class": "css-164r41r"})
    
    # extracting baths, rooms, parking etc
    all_basic_features = bs_object.find("div", {"class": "css-ghc6s4"}).findAll("span", {"data-testid": "property-features-text-container"})
    
    # extracting property price
    property_price = bs_object.find("div", {"data-testid": "listing-details__summary-title"})
    
    # extracting latitudes and longitudes
    lat_long = bs_object.find("a", {"target": "_blank", 'rel': "noopener noreferer"})
    
    #extract schoolname, school distance
    school= bs_object.find("div", {"data-testid": "listing-details__school-catchment"}).findAll("h5",{"class": "css-5w5cop"})
    school_distance= bs_object.find("div", {"data-testid": "listing-details__school-catchment"}).findAll("div", {"data-testid": "fe-co-school-catchment-schoolDistance"})
  
    # address info
    address1 = bs_object.find("span", {"data-testid": "address-line1"})
    address2 = bs_object.find("span", {"data-testid": "address-line2"})
    
    #recent rented
    recent_rented_number = bs_object.find("div", {"class": "css-12uo2x5"})
    
    # dictionary to store temporary data
    basic_feature_dict = {}
    
    # few properties does not contain all the 4 features such as rooms, baths, parkings, area. So need to check
    # how many features they contain
    if len(all_basic_features) == 4:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[2]))[0][1]] = pattern.findall(str(all_basic_features[2]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[3]))[0][1]] = pattern.findall(str(all_basic_features[3]))[0][0]
        
    elif len(all_basic_features) == 3:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[2]))[0][1]] = pattern.findall(str(all_basic_features[2]))[0][0]
        
    elif len(all_basic_features) == 2:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        
    elif len(all_basic_features) == 1:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]

        
        
    # putting 'none' if school is missing    
    if school is None:
        basic_feature_dict['school'] = None
        
    else:
        basic_feature_dict['school'] = sum([pattern_school.findall(str(i)) for i in school],[])
    
    # putting 'none' if school distance is missing   
    if school_distance is None:
        basic_feature_dict['school_distance'] = None
        
    else:
        basic_feature_dict['school_distance'] = sum([pattern_distance.findall(str(i)) for i in school_distance],[])
     
    
    
    # putting 'none' if address(street) distance is missing   
    if address1 is None:
        basic_feature_dict['address1'] = None
        
    else:
        basic_feature_dict['address1'] = pattern_address1.findall(str(address1))
        
    # putting 'none' if address2 is missing   
    if address2 is None:
        basic_feature_dict['address2'] = None
        
    else:
        basic_feature_dict['address2'] = sum([pattern_address2.findall(str(i)) for i in address2],[])
    
        # putting 'none' if recent_rented number is missing   
    if recent_rented_number is None:
        basic_feature_dict['recent_rented_number'] = None
        
    else:
        basic_feature_dict['recent_rented_number'] = pattern1.findall(str(recent_rented_number))[0]


    # putting 'none' if price is missing    
    if property_price is None:
        basic_feature_dict['price'] = None
        
    else:
        basic_feature_dict['price'] = pattern1.findall(str(property_price))[0]
        
    # putting 'none' if property name/address is missing       
    if property_name is None:
        basic_feature_dict['name'] = None
        
    else:
        basic_feature_dict['name'] = pattern1.findall(str(property_name))[0]
        
    # putting 'none' if latitude and logitude are missing        
    if lat_long is None:
        basic_feature_dict['lat'] = None
        basic_feature_dict['long'] = None
        
    else:
        basic_feature_dict['lat'] = pattern2.findall(str(lat_long))[0].split(',')[0]
        basic_feature_dict['long'] = pattern2.findall(str(lat_long))[0].split(',')[1]
        
    # appending all the data into a list
    basic_feature_list.append(basic_feature_dict)

In [9]:
# creating empty lists
beds_list = []
baths_list = []
parking_list = []
area_list = []
name_list = []
lat_list = []
long_list = []
price_list = []
school_list = []
school_distance_list =[]
address1_list = []
address2_list = []
recent_rented_number_list = []
recent_rented_price_list = []

# interating through list created above with data
for row in basic_feature_list:
    
    # checking if the row cointains 'Beds', 'Bed' or nothing
    if 'Beds' in row:
        beds_list.append(row['Beds'])
    elif 'bed' in row:
        beds_list.append(row['Bed'])
    else:
        beds_list.append(None)
        
    # checking if the row cointains 'Baths', 'Bath' or nothing    
    if 'Baths' in row:
        baths_list.append(row['Baths'])
    elif 'Bath ' in row:
        baths_list.append(row['Bath'])
    else:
        baths_list.append(None)
        
    # checking if the row cointains 'Parking', '-' or nothing     
    if 'Parking' in row and row['Parking'] != '−':
        parking_list.append(row['Parking'])
    else:
        parking_list.append(None)
        
    # checking if the row cointains ' ', or nothing. Because empty space (i.e. ' ') reprsents area  
    if ' ' in row:
        area_list.append(row[' '])
    else:
        area_list.append(None)
    # checking if the row cointains 'name' that is address of property         
    if 'name' in row:
        name_list.append(row['name'])
    else:
        name_list.append(None)
    
    # checking if the row cointains 'price'         
    if 'price' in row:
        price_list.append(row['price'])
    else:
        price_list.append(None)        
    
    # checking if the row cointains 'lat' that is lattitude of property         
    if 'lat' in row:
        lat_list.append(row['lat'])
    else:
        lat_list.append(None)  
        
    # checking if the row cointains 'long' that is lattitude of property             
    if 'long' in row:
        long_list.append(row['long'])
    else:
        long_list.append(None)
        
    # checking if the row cointains 'school' that is lattitude of property             
    if 'school' in row:
        school_list.append(row['school'])
    else:
        school_list.append(None)
        
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'school_distance' in row:
        school_distance_list.append(row['school_distance'])
    else:
        school_distance_list.append(None)
        
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'address1' in row:
        address1_list.append(row['address1'])
    else:
        address1_list.append(None)
        
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'address2' in row:
        address2_list.append(row['address2'])
    else:
        address2_list.append(None)
        
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'recent_rented_number' in row:
        recent_rented_number_list.append(row['recent_rented_number'])
    else:
        recent_rented_number_list.append(None)
    
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'recent_rented_price' in row:
        recent_rented_price_list.append(row['recent_rented_price'])
    else:
        recent_rented_price_list.append(None)

In [10]:
#create dataframe 
house_dict = {}
house_dict['Beds'] = beds_list
house_dict['Baths'] = baths_list
house_dict['Parking'] = parking_list
house_dict['Area'] = area_list
house_dict['Address'] = name_list
house_dict['Latitude'] = lat_list
house_dict['Longitude'] = long_list
house_dict['Rent'] = price_list
house_dict['School'] = school_list
house_dict['School_distance'] = school_distance_list
house_dict['Location_1'] = address1_list
house_dict['Location_2'] = address2_list
house_dict['recent_rented_number'] = recent_rented_number_list
house_dict['recent_rented_price'] = recent_rented_price_list
house_df = pd.DataFrame(house_dict)

In [11]:
def remove_chars(s):
    return re.sub('[^0-9.]+', '', s) 

def remove_squaremeter(s):
    if s != None:
        return remove_chars(s)
    else:
        return None

In [12]:
#find pattern for week/annual rent
year_pattern = r"p.a.|annually|annual"
week_pattern = r"pw|week"

regex_week_pattern = re.compile(week_pattern)
regex_year_pattern = re.compile(year_pattern)

#flag type of rent with 1(weekly), 0(annul), None(unknown)
def type_flag(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return 1
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return 0
    # not speficied
    else:
        return None

#convert rent to weekly rent
#if not sure rent type, it will be considered as weekly rent
def convert_rent(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return remove_chars(s)
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return float(remove_chars(s))//12
    # not speficied
    else:
        return remove_chars(s)


house_df['Converted_Rent'] = [convert_rent(i) for i in price_list]
house_df['Type'] = [type_flag(i) for i in price_list]

In [13]:
#extract post code
house_df['Postcode'] = house_df['Address'].str[-4:]

#clean area
house_df['Area'] = [remove_squaremeter(i) for i in area_list]

In [14]:
house_df

,Beds,Baths,Parking,Area,Address,Latitude,Longitude,Rent,School,School_distance,Location_1,Location_2,recent_rented_number,recent_rented_price,Converted_Rent,Type,Postcode
0,None,None,1,None,CARPARK/442 Elizabeth Street Melbourne VIC 3000,-37.8083542,144.9607272,$60 per week,"[Carlton Gardens Primary School, University Hi...","[1 km, 1.3 km, 0.5 km]",None,None,None,None,60,1.0,3000
1,None,None,None,None,3405/288 Spencer Street Melbourne VIC 3000,-37.813775,144.9520948,$440,"[University High School, Docklands Primary Sch...","[1.9 km, 0.9 km, 1.7 km]",[288 Spencer St],"[Melbourne, VIC, 3000]",315,None,440,NaN,3000
2,None,None,None,None,1007/238 Flinders St Melbourne VIC 3000,-37.8175167,144.9664983,$320,"[Carlton Gardens Primary School, University Hi...","[1.7 km, 2.5 km, 0.8 km]",[238 Flinders St],"[Melbourne, VIC, 3000]",255,None,320,NaN,3000
3,2,2,None,None,1211/260 Spencer Street Melbourne VIC 3000,-37.814564,144.9522856,$700,"[University High School, Docklands Primary Sch...","[2 km, 1 km, 1.6 km]",[260 Spencer St],"[Melbourne, VIC, 3000]",671,None,700,NaN,3000
4,2,None,None,None,504/350 La Trobe Street Melbourne VIC 3000,-37.8106785,144.9592774,$395 per week,"[Carlton Gardens Primary School, University Hi...","[1.3 km, 1.6 km, 0.2 km]",[350 La Trobe St],"[Melbourne, VIC, 3000]",71,None,395,1.0,3000
5,None,None,None,None,1106/555 Flinders Street Melbourne VIC 3000,-37.8209711,144.9564827,$370 per week,"[Port Phillip Specialist School, University Hi...","[2.2 km, 2.7 km, 0.9 km]",[555 Flinders St],"[Melbourne, VIC, 3000]",465,None,370,1.0,3000
6,2,None,1,None,2901/200 Spencer Street Melbourne VIC 3000,-37.816228,144.9532465,$500 per week,"[University High School, Docklands Primary Sch...","[2.1 km, 1.1 km, 1.4 km]",[200 Spencer St],"[Melbourne, VIC, 3000]",506,None,500,1.0,3000
7,3,3,2,None,5401/464 Collins Street Melbourne VIC 3000,-37.8175602,144.9585916,$2500 Per Week,"[University High School, South Melbourne Prima...","[2.3 km, 1.4 km, 1.6 km]",[464 Collins St],"[Melbourne, VIC, 3000]",289,None,2500,1.0,3000
8,None,None,None,None,405/39 Queen Street Melbourne VIC 3000,-37.817888,144.9617372,$450.00 per week,"[University High School, South Melbourne Prima...","[2.4 km, 1.4 km, 1.9 km]",[39 Queen St],"[Melbourne, VIC, 3000]",61,None,450.00,1.0,3000
9,2,3,2,None,113/350 St Kilda Road Melbourne VIC 3004,-37.831142,144.9708545,$1500 Per Week,"[South Melbourne Park Primary School, Port Phi...","[1.3 km, 2.6 km, 2.9 km]",[350 St Kilda Rd],"[Melbourne, VIC, 3004]",79,None,1500,1.0,3004


In [28]:
bs_object = BeautifulSoup(
    requests.get("https://www.domain.com.au/1-watson-street-armadale-vic-3143-16061315",
                 headers=headers).text, "html.parser")

In [30]:
school= bs_object.find("div", {"data-testid": "listing-details__school-catchment"}).findAll("h5",{"class": "css-5w5cop"})
school

[<h5 class="css-5w5cop" data-testid="fe-co-school-catchment-school-title" itemprop="name">Armadale Primary School</h5>,
 <h5 class="css-5w5cop" data-testid="fe-co-school-catchment-school-title" itemprop="name">Auburn High School</h5>,
 <h5 class="css-5w5cop" data-testid="fe-co-school-catchment-school-title" itemprop="name">Port Phillip Specialist School</h5>]

In [33]:
price = bs_object.find("div", {"data-testid": "listing-details__school-catchment"})
price_value = ''.join([x.text for x in bs_object.find_all("h5",{"class": "css-5w5cop"})])
print(price_value)

Armadale Primary SchoolAuburn High SchoolPort Phillip Specialist School
